### Primera versión de prueba de conexión gdrive

NO CORRER

In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Definir el scope
SCOPES = ['https://www.googleapis.com/auth/drive.file']

# Ruta al archivo de credenciales descargado
CREDENTIALS_FILE = './credenciales/client_secret_714287611170-vgk4sfc7gqhg9monrjgc6hf66r74cqfj.apps.googleusercontent.com.json'

def authenticate_gdrive():
    flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
    creds = flow.run_local_server(port=0)  # Autenticación mediante un servidor local
    return creds

# Construye el servicio de Google Drive utilizando las credenciales
service = build('drive', 'v3', credentials=authenticate_gdrive())

# Ahora puedes usar 'service' para interactuar con Google Drive

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=714287611170-vgk4sfc7gqhg9monrjgc6hf66r74cqfj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A48883%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&state=588sr34zPWPv3TTPAg4jK6B7TX7SEA&access_type=offline
Abriendo en una sesión existente del navegador


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/hugo/snap/code/common/.cache/gio-modules/libgiolibproxy.so


### Versión que autentica, crea carpera y sube

Sin control si lo que hay está subido

NO CORRER

In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.http import MediaFileUpload
import os

# Definir el scope
SCOPES = ['https://www.googleapis.com/auth/drive']
CREDENTIALS_FILE = './credenciales/client_secret_714287611170-vgk4sfc7gqhg9monrjgc6hf66r74cqfj.apps.googleusercontent.com.json'

def authenticate_gdrive():
    flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
    creds = flow.run_local_server(port=0)
    return build('drive', 'v3', credentials=creds)

def create_or_find_folder(service, folder_name, parent_id=None):
    query = f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}'"
    if parent_id:
        query += f" and '{parent_id}' in parents"
    results = service.files().list(q=query, spaces='drive', fields="files(id, name)").execute()
    folders = results.get('files', [])

    if not folders:
        folder_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder',
            'parents': [parent_id] if parent_id else []
        }
        folder = service.files().create(body=folder_metadata, fields='id').execute()
        return folder.get('id')
    return folders[0].get('id')

def upload_files(service, folder_path, drive_folder_id):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            file_metadata = {
                'name': filename,
                'parents': [drive_folder_id]
            }
            media = MediaFileUpload(file_path, resumable=True)
            file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
            print(f'Uploaded {filename} to Google Drive with File ID: {file.get("id")}')


### Versión final

Autentica, crea carpeta y genera el servicio

In [2]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.http import MediaFileUpload
import os

# Definir el scope
SCOPES = ['https://www.googleapis.com/auth/drive.file']
CREDENTIALS_FILE = './credenciales/client_secret_714287611170-vgk4sfc7gqhg9monrjgc6hf66r74cqfj.apps.googleusercontent.com.json'

def authenticate_gdrive():
    flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
    creds = flow.run_local_server(port=0)
    return build('drive', 'v3', credentials=creds)

def create_or_find_folder(service, folder_name, parent_id=None):
    query = f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}'"
    if parent_id:
        query += f" and '{parent_id}' in parents"
    results = service.files().list(q=query, spaces='drive', fields="files(id, name)").execute()
    folders = results.get('files', [])

    if not folders:
        folder_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder',
            'parents': [parent_id] if parent_id else []
        }
        folder = service.files().create(body=folder_metadata, fields='id').execute()
        return folder.get('id')
    return folders[0].get('id')


service = authenticate_gdrive()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=714287611170-vgk4sfc7gqhg9monrjgc6hf66r74cqfj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A36881%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&state=UtWzmEHt6V9pG0v5MYrZsGFExsTnGc&access_type=offline
Abriendo en una sesión existente del navegador


/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/hugo/snap/code/common/.cache/gio-modules/libgiolibproxy.so


### Guardar el objeto del "servicio"

In [10]:
import pickle
# Guardar las credenciales en un archivo
with open('token.pkl', 'wb') as token:
    pickle.dump(service, token)

### Compruba si el doc existe y sube solo documentos nuevos

In [4]:
def file_exists(service, folder_id, file_name):
    """Check if a file exists in the specified Google Drive folder."""
    query = f"'{folder_id}' in parents and name = '{file_name}'"
    results = service.files().list(q=query, spaces='drive', fields="files(id, name)").execute()
    files = results.get('files', [])
    return files[0]['id'] if files else None

def upload_files(service, folder_path, drive_folder_id):
    """Upload files to Google Drive, only if they do not already exist."""
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            # Check if the file already exists in Google Drive
            existing_file_id = file_exists(service, drive_folder_id, filename)
            if existing_file_id:
                print(f'Skipping {filename}, already exists with File ID: {existing_file_id}')
                continue  # Skip this file as it already exists

            # If the file does not exist, upload it
            file_metadata = {
                'name': filename,
                'parents': [drive_folder_id]
            }
            media = MediaFileUpload(file_path, resumable=True)
            file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
            print(f'Uploaded {filename} to Google Drive with File ID: {file.get("id")}')


### Función de subida de datos

In [5]:
def main():
    
    local_folder_path = './datos'
    drive_folder_name = 'tp_textminig/datos_csv'

    # Encuentra o crea la carpeta en Drive
    drive_folder_id = create_or_find_folder(service, drive_folder_name)

    # Sube los archivos
    upload_files(service, local_folder_path, drive_folder_id)

if __name__ == '__main__':
    main()

Skipping 2024-05-15_noticias_economicas_internacionales.csv, already exists with File ID: 112lazUhYZnjg5Zv7JVMgvhPrsGGTkgWM
Skipping 2024-05-14_noticias_economicas_nacionales.csv, already exists with File ID: 1C0Aj4n6nVQNhCodK6krdxrwGtVcfUXmv
Skipping 2024-05-16_noticias_economicas_nacionales.csv, already exists with File ID: 1yAIOFU97TvI1ogbtE5CBrtZVDwqKKVW7
Skipping 2024-05-16_noticias_economicas_internacionales.csv, already exists with File ID: 11XnDDENj0YKwhCtM_Q25iE7_tx_GcwZt
Skipping 2024-05-13_noticias_economicas_nacionales.csv, already exists with File ID: 1cha7MQKdSFcUbXU-fqNITsNu1iRHiCPv
Skipping 2024-05-13_noticias_economicas_internacionales.csv, already exists with File ID: 1lMM-otypwgsO2ImcgdHH8iP4ObxEpD6H
Skipping 2024-05-14_noticias_economicas_internacionales.csv, already exists with File ID: 17S8s2IFH2iKsD9XxiOm-I5IqDcvDWBIK
Skipping 2024-05-15_noticias_economicas_nacionales.csv, already exists with File ID: 15xYXzDu8Bp2w36tumANfvGg11-QDU9XP
Uploaded clasificacion.csv t